In [7]:

import pandas as pd

batting_data = pd.read_csv("batting_data.csv")

print(batting_data.head())

batting_data['Performance_Index'] = (
    batting_data['Swing_Speed'] * 0.4 +                   
    batting_data['Reaction_Time'] * 0.3 +        
    batting_data['Grip_Pressure'] * 0.2 +               
    batting_data['Heart_Rate'] * 0.1            
)

print(batting_data.head())

   Heart_Rate  Reaction_Time  Swing_Speed  Grip_Pressure
0  128.532180     423.323502    22.500225      17.538166
1  152.260358     364.527591    31.381385      22.089555
2  100.397536     386.137502    31.996400      46.039986
3  121.359507     344.419954    36.371314      25.125513
4  126.278983     365.849088    24.031426      29.957395
   Heart_Rate  Reaction_Time  Swing_Speed  Grip_Pressure  Performance_Index
0  128.532180     423.323502    22.500225      17.538166         152.357992
1  152.260358     364.527591    31.381385      22.089555         141.554778
2  100.397536     386.137502    31.996400      46.039986         147.887561
3  121.359507     344.419954    36.371314      25.125513         135.035565
4  126.278983     365.849088    24.031426      29.957395         137.986674
